# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient
db = 'companies'
client = MongoClient(f"mongodb://localhost/{db}")
db = client.get_database()

In [4]:
# Your Code
cur = db.list_collections()
for collection in cur:
    print(collection)

{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('76992dc7-b5e1-4dcd-9a37-b36c0e165615')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'companies.companies'}}


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [9]:
cur = db.companies.find({"name":"Babelgum"})
for collec in cur:
    print(collec["name"])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [18]:
from pymongo import ASCENDING, DESCENDING
cur = db.companies.find({"number_of_employees":{ "$gt" :5000}}
                       ).sort([("number_of_employees", DESCENDING)]
                        ).limit(20)
for collec in cur:
    print( collec["number_of_employees"], collec["name"])

405000 Siemens
388000 IBM
320000 Toyota
300000 PayPal
227000 Nippon Telegraph and Telephone Corporation
221726 Samsung Electronics
205000 Accenture
200300 Tata Consultancy Services
200000 Flextronics International
186000 Safeway
180500 Sony
177000 LG
171000 Ford
160000 Boeing
140000 Digital Equipment Corporation
125000 Nokia
107000 MItsubishi Electric
107000 MItsubishi Electric
100000 Comcast
100000 Bertelsmann


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [59]:
cur = db.companies.find({"founded_year":{"$gte":2000, "$lte":2005}}).count()

# comment out the long answer, and only querying the count()
#for coll in cur:
 #   print(coll['founded_year'], coll['name'], )
cur

/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


3734

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [147]:
# Your Code
cur = db.companies.find({"founded_year": {"$lt":2010},
                         "ipo.valuation_amount":{"$gt":100000000}
                        }).limit(3)

for coll in cur:
    print(coll['name'], coll['ipo'])

Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Twitter {'valuation_amount': 18100000000, 'valuation_currency_code': 'USD', 'pub_year': 2013, 'pub_month': 11, 'pub_day': 7, 'stock_symbol': 'NYSE:TWTR'}
Yelp {'valuation_amount': 1300000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 3, 'pub_day': 2, 'stock_symbol': 'NYSE:YELP'}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [55]:
# Your Code
cur = db.companies.find({"number_of_employees":{"$lt":1000}, 
                         "founded_year":{"$lt":2005}
                        },{
                           "founded_year":1,"name":1, "number_of_employees":1,
                       }).limit(10).sort([("number_of_employees",DESCENDING)])

for x in list(cur):
    print(x["number_of_employees"], x["founded_year"],x['name'],"\t",)
    
# OK CREO QUE ES MUCHO MAS INTUITIVO QUE MYSQL TAMBIEN, AL MENOS A LA HORA DE QUERY

974 2000 Infinera Corporation 	
948 1997 NorthPoint Communications Group 	
931 1997 888 Holdings 	
903 1983 Forrester Research 	
900 1991 SonicWALL 	
900 1999 Webmetrics 	
881 1999 Cornerstone OnDemand 	
800 2004 Yelp 	
800 2000 ZoomInfo 	
800 2003 MySpace 	


### 6. All the companies that don't include the `partners` field.

In [66]:
cur1 = db.companies.find({"partners":{"$exists":True}}).count()
cur2 = db.companies.find({"partners":{"$exists":False}}).count()
cur1, cur2

/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.
/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


(18801, 0)

### 7. All the companies that have a null type of value on the `category_code` field.

In [148]:
db.companies.find({"catergory_code":{"$type":"null"}}).count()

/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


0

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [171]:
# Your Code
c =db.companies.find({"number_of_employees":{"$gt":100, "$lt":1000}},
                     {"number_of_employees":1,"name":1}
                    ).sort([("number_of_employees",-1)]).limit(10)
for c in c:
    print(c["number_of_employees"],c["name"])

984 Datamonitor
974 Infinera Corporation
950 Box
948 NorthPoint Communications Group
931 888 Holdings
903 Forrester Research
900 Workday
900 InMobi
900 Relax Solutions Pvt Ltd
900 SonicWALL


### 9. Order all the companies by their IPO price in a descending order.

In [194]:
# ¿Por que no funciona el query usando el operador $not  con $type??

c=db.companies.find({"ipo.valuation_amount":  {"$not":  {"$type":10}  }
                    }, {"name":1,"ipo.valuation_amount":1}
                   ) #.sort("ipo.valuation_amount",-1)
                   
for e in c:
    print(e) #,e["ipo.valuation_amount"])

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'), 'name': 'AdventNet'}
{'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'name': 'Wetpaint'}
{'_id': ObjectId('52cdef7c4bab8bd675297d8c'), 'name': 'Zoho'}
{'_id': ObjectId('52cdef7c4bab8bd675297d8f'), 'name': 'Omnidrive'}
{'_id': ObjectId('52cdef7c4bab8bd675297d91'), 'name': 'Geni'}
{'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'}
{'_id': ObjectId('52cdef7c4bab8bd675297d8d'), 'name': 'Digg'}
{'_id': ObjectId('52cdef7c4bab8bd675297d92'), 'name': 'Flektor'}
{'_id': ObjectId('52cdef7c4bab8bd675297d93'), 'name': 'Fox Interactive Media'}
{'_id': ObjectId('52cdef7c4bab8bd675297d96'), 'name': 'Gizmoz'}
{'_id': ObjectId('52cdef7c4bab8bd675297d97'), 'name': 'Scribd'}
{'_id': ObjectId('52cdef7c4bab8bd675297d98'), 'name': 'Slacker'}
{'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}}
{'_id': ObjectId('52cdef7c4bab8bd675297d99'), 'name': 'Lala'}
{'_id': ObjectId('52cdef7c4bab8bd675297d9

{'_id': ObjectId('52cdef7c4bab8bd675297fe6'), 'name': 'Metaplace'}
{'_id': ObjectId('52cdef7c4bab8bd675297fe8'), 'name': 'CafeScribe'}
{'_id': ObjectId('52cdef7c4bab8bd675297fe9'), 'name': 'StoryBlender'}
{'_id': ObjectId('52cdef7c4bab8bd675297fea'), 'name': 'Spoke'}
{'_id': ObjectId('52cdef7c4bab8bd675297feb'), 'name': 'BroadClip'}
{'_id': ObjectId('52cdef7c4bab8bd675297fec'), 'name': 'Fixya'}
{'_id': ObjectId('52cdef7c4bab8bd675297fef'), 'name': 'Wirenode'}
{'_id': ObjectId('52cdef7c4bab8bd675297fee'), 'name': 'Tripwiser'}
{'_id': ObjectId('52cdef7c4bab8bd675297fed'), 'name': 'Myxer'}
{'_id': ObjectId('52cdef7c4bab8bd675297ff1'), 'name': 'Syntraxis'}
{'_id': ObjectId('52cdef7c4bab8bd675297ff0'), 'name': 'Mozilla'}
{'_id': ObjectId('52cdef7c4bab8bd675297ff3'), 'name': 'BuzzDash'}
{'_id': ObjectId('52cdef7c4bab8bd675297ff4'), 'name': 'BigSwerve'}
{'_id': ObjectId('52cdef7c4bab8bd675297ff2'), 'name': 'BrightQube'}
{'_id': ObjectId('52cdef7c4bab8bd675297ff5'), 'name': 'Styky'}
{'_id': Ob

{'_id': ObjectId('52cdef7c4bab8bd67529880c'), 'name': 'Patents from RU'}
{'_id': ObjectId('52cdef7c4bab8bd67529880d'), 'name': 'BMAT'}
{'_id': ObjectId('52cdef7c4bab8bd67529880e'), 'name': 'GoLearnTo'}
{'_id': ObjectId('52cdef7c4bab8bd675298811'), 'name': 'Mercury Intermedia'}
{'_id': ObjectId('52cdef7c4bab8bd675298810'), 'name': 'SOSIUS'}
{'_id': ObjectId('52cdef7c4bab8bd675298812'), 'name': 'AdXpose'}
{'_id': ObjectId('52cdef7c4bab8bd675298814'), 'name': 'Froogle'}
{'_id': ObjectId('52cdef7c4bab8bd67529880f'), 'name': 'Televisa'}
{'_id': ObjectId('52cdef7c4bab8bd675298816'), 'name': 'IceChat'}
{'_id': ObjectId('52cdef7c4bab8bd675298815'), 'name': 'mIRC'}
{'_id': ObjectId('52cdef7c4bab8bd675298813'), 'name': 'BizRate'}
{'_id': ObjectId('52cdef7c4bab8bd675298817'), 'name': 'Mibbit'}
{'_id': ObjectId('52cdef7c4bab8bd67529881a'), 'name': 'Vendare Media'}
{'_id': ObjectId('52cdef7c4bab8bd675298818'), 'name': 'MyFab'}
{'_id': ObjectId('52cdef7c4bab8bd67529881b'), 'name': 'Twitlinks'}
{'_id

{'_id': ObjectId('52cdef7d4bab8bd675298e6f'), 'name': 'Bazaar Advertising Solutions'}
{'_id': ObjectId('52cdef7d4bab8bd675298e6d'), 'name': 'quimble'}
{'_id': ObjectId('52cdef7d4bab8bd675298e6e'), 'name': 'Quizilla'}
{'_id': ObjectId('52cdef7d4bab8bd675298e71'), 'name': 'LogicalApps'}
{'_id': ObjectId('52cdef7d4bab8bd675298e72'), 'name': 'Flashbase'}
{'_id': ObjectId('52cdef7d4bab8bd675298e70'), 'name': 'Informative'}
{'_id': ObjectId('52cdef7d4bab8bd675298e73'), 'name': 'Media Bloggers Association'}
{'_id': ObjectId('52cdef7d4bab8bd675298e74'), 'name': 'Lolz'}
{'_id': ObjectId('52cdef7d4bab8bd675298e75'), 'name': 'Advantia VoIP Systems'}
{'_id': ObjectId('52cdef7d4bab8bd675298e76'), 'name': 'Uniblue'}
{'_id': ObjectId('52cdef7d4bab8bd675298e79'), 'name': 'MYSTYLEPOST'}
{'_id': ObjectId('52cdef7d4bab8bd675298e78'), 'name': 'Jogli'}
{'_id': ObjectId('52cdef7d4bab8bd675298e77'), 'name': 'GrowVotes'}
{'_id': ObjectId('52cdef7d4bab8bd675298e7a'), 'name': 'Forcestream'}
{'_id': ObjectId('52

{'_id': ObjectId('52cdef7d4bab8bd675299430'), 'name': 'Communitech'}
{'_id': ObjectId('52cdef7d4bab8bd67529942f'), 'name': 'ParkVu'}
{'_id': ObjectId('52cdef7d4bab8bd675299431'), 'name': 'MeterNet'}
{'_id': ObjectId('52cdef7d4bab8bd675299433'), 'name': 'ImThere'}
{'_id': ObjectId('52cdef7d4bab8bd675299432'), 'name': 'Albion'}
{'_id': ObjectId('52cdef7d4bab8bd675299434'), 'name': 'swindlr'}
{'_id': ObjectId('52cdef7d4bab8bd675299436'), 'name': 'Don Russell Family Investments'}
{'_id': ObjectId('52cdef7d4bab8bd675299435'), 'name': 'Doodle'}
{'_id': ObjectId('52cdef7d4bab8bd675299438'), 'name': 'CODA'}
{'_id': ObjectId('52cdef7d4bab8bd675299437'), 'name': 'LeisureLogix'}
{'_id': ObjectId('52cdef7d4bab8bd67529942d'), 'name': 'Covarity'}
{'_id': ObjectId('52cdef7d4bab8bd67529943a'), 'name': 'Hihera'}
{'_id': ObjectId('52cdef7d4bab8bd67529942c'), 'name': 'JupiterResearch'}
{'_id': ObjectId('52cdef7d4bab8bd67529943b'), 'name': 'InstantBull'}
{'_id': ObjectId('52cdef7d4bab8bd675299439'), 'name

...

{'_id': ObjectId('52cdef7e4bab8bd67529ac4d'), 'name': '6th Sense Analytics'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac4c'), 'name': 'id Software'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac4f'), 'name': 'MonoSphere'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac50'), 'name': 'KlickSports'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac52'), 'name': 'Sina ADI'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac51'), 'name': 'UpClick Marketing'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac4a'), 'name': 'Lottay'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac4e'), 'name': 'Voxer LLC'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac55'), 'name': 'Triforma - Innotiimi'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac54'), 'name': 'Ceylon Solutions'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac56'), 'name': 'Generate Partners'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac58'), 'name': 'Rudius Media'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac53'), 'name': 'D2 Technologies'}
{'_id': ObjectId('52cdef7e4bab8bd67529ac57'), 'name': 'TradeUrban'}
{'_id':

{'_id': ObjectId('52cdef7e4bab8bd67529b301'), 'name': 'UnHub'}
{'_id': ObjectId('52cdef7e4bab8bd67529b302'), 'name': 'Youth Bloggers Network'}
{'_id': ObjectId('52cdef7e4bab8bd67529b303'), 'name': 'Point Motive'}
{'_id': ObjectId('52cdef7e4bab8bd67529b305'), 'name': 'Origramy'}
{'_id': ObjectId('52cdef7e4bab8bd67529b306'), 'name': 'rent2friend'}
{'_id': ObjectId('52cdef7e4bab8bd67529b304'), 'name': 'Guava UK'}
{'_id': ObjectId('52cdef7e4bab8bd67529b309'), 'name': 'UQ Events'}
{'_id': ObjectId('52cdef7e4bab8bd67529b307'), 'name': 'TheOfficialBoard'}
{'_id': ObjectId('52cdef7e4bab8bd67529b308'), 'name': 'VideoCustomizer'}
{'_id': ObjectId('52cdef7e4bab8bd67529b30c'), 'name': 'i-Info'}
{'_id': ObjectId('52cdef7e4bab8bd67529b2e6'), 'name': 'Cucumis'}
{'_id': ObjectId('52cdef7e4bab8bd67529b30a'), 'name': 'Wolfram Alpha'}
{'_id': ObjectId('52cdef7e4bab8bd67529b30b'), 'name': 'MIR3'}
{'_id': ObjectId('52cdef7e4bab8bd67529b30f'), 'name': 'CompareChecker'}
{'_id': ObjectId('52cdef7e4bab8bd67529

{'_id': ObjectId('52cdef7e4bab8bd67529b88e'), 'name': 'MegaShare'}
{'_id': ObjectId('52cdef7e4bab8bd67529b88c'), 'name': 'Greentree'}
{'_id': ObjectId('52cdef7e4bab8bd67529b88a'), 'name': 'CIOpedia'}
{'_id': ObjectId('52cdef7e4bab8bd67529b88f'), 'name': 'VH1'}
{'_id': ObjectId('52cdef7e4bab8bd67529b890'), 'name': 'Infinity Broadcasting'}
{'_id': ObjectId('52cdef7e4bab8bd67529b893'), 'name': 'ITT Industries'}
{'_id': ObjectId('52cdef7e4bab8bd67529b892'), 'name': 'Flowserve'}
{'_id': ObjectId('52cdef7e4bab8bd67529b896'), 'name': 'Catchafire'}
{'_id': ObjectId('52cdef7e4bab8bd67529b897'), 'name': 'TunedSystems'}
{'_id': ObjectId('52cdef7e4bab8bd67529b898'), 'name': 'MicroGen Systems'}
{'_id': ObjectId('52cdef7e4bab8bd67529b895'), 'name': 'Novaled'}
{'_id': ObjectId('52cdef7e4bab8bd67529b899'), 'name': 'Ingeniux'}
{'_id': ObjectId('52cdef7e4bab8bd67529b894'), 'name': 'Rentricity'}
{'_id': ObjectId('52cdef7e4bab8bd67529b89a'), 'name': 'Credentify'}
{'_id': ObjectId('52cdef7e4bab8bd67529b891

{'_id': ObjectId('52cdef7f4bab8bd67529be0b'), 'name': 'ACT Biotech'}
{'_id': ObjectId('52cdef7f4bab8bd67529be0d'), 'name': 'Actimis Pharmaceuticals'}
{'_id': ObjectId('52cdef7f4bab8bd67529be11'), 'name': 'Melville Candy'}
{'_id': ObjectId('52cdef7f4bab8bd67529be0f'), 'name': 'Innovative Biosensors'}
{'_id': ObjectId('52cdef7f4bab8bd67529be12'), 'name': 'Mobile Middleware'}
{'_id': ObjectId('52cdef7f4bab8bd67529be13'), 'name': 'Informatix'}
{'_id': ObjectId('52cdef7f4bab8bd67529be10'), 'name': '3x3Links'}
{'_id': ObjectId('52cdef7f4bab8bd67529be14'), 'name': 'Ampergen'}
{'_id': ObjectId('52cdef7f4bab8bd67529be15'), 'name': 'Latamdata'}
{'_id': ObjectId('52cdef7f4bab8bd67529be0e'), 'name': 'Apieron'}
{'_id': ObjectId('52cdef7f4bab8bd67529be19'), 'name': 'Solace Systems'}
{'_id': ObjectId('52cdef7f4bab8bd67529be17'), 'name': '888 Holdings'}
{'_id': ObjectId('52cdef7f4bab8bd67529be18'), 'name': 'ImaHima'}
{'_id': ObjectId('52cdef7f4bab8bd67529be1b'), 'name': 'Transarc'}
{'_id': ObjectId('5

{'_id': ObjectId('52cdef7f4bab8bd67529c3a4'), 'name': 'Telogis'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3aa'), 'name': 'Three Pillar Software'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3a8'), 'name': 'WKD Solutions'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3a9'), 'name': 'Firefly Database Solutions'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3ab'), 'name': 'Internetseer'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3ac'), 'name': 'Mind Drivers'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3a5'), 'name': 'Force10 Networks'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3ad'), 'name': 'Certified CIO'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3ae'), 'name': 'Kear IT'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3b0'), 'name': 'Bluestone Energy'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3b2'), 'name': 'Educational Options'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3b1'), 'name': 'Siboney Learning Group'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3b3'), 'name': 'Edgenuity'}
{'_id': ObjectId('52cdef7f4bab8bd67529c3b5'), 'name':

In [191]:
c=db.companies.find({"ipo.valuation_amount":  {"$ne":  None  }
                    }, {"name":1,"ipo.valuation_amount":1}
                   ).sort("ipo.valuation_amount",-1).limit(10)
                   
for e in c:
    print(e["ipo"]["valuation_amount"], e["name"])

108960000000 GREE
104000000000 Facebook
100000000000 Amazon
18100000000 Twitter
12800000000 Groupon
11000000000 Tencent
9430000000 Western Digital
9310000000 LinkedIn
6000000000 BMC Software
5440000000 Rackspace


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [86]:
# Your Code
list(db.companies.find({},    {"number_of_employees":1, "name":1 }
                      ).sort([("number_of_employees",DESCENDING)]).limit(10))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'number_of_employees': 405000},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'),
  'name': 'IBM',
  'number_of_employees': 388000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'),
  'name': 'Toyota',
  'number_of_employees': 320000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'_id': ObjectId('52cdef7d4bab8bd675298aa4'),
  'name': 'Samsung Electronics',
  'number_of_employees': 221726},
 {'_id': ObjectId('52cdef7d4bab8bd675298b99'),
  'name': 'Accenture',
  'number_of_employees': 205000},
 {'_id': ObjectId('52cdef7e4bab8bd67529a657'),
  'name': 'Tata Consultancy Services',
  'number_of_employees': 200300},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'),
  'name': 'Flextronics International',
  'number_of_employees': 200000},
 {'

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [198]:
c = db.companies.find({"founded_month":{"$in":[e for e in range(7,13)]}}
                     ).limit(10)
for coll in c:
    print(coll["founded_month"], coll['name'],)

10 Wetpaint
9 Zoho
11 Omnidrive
10 Digg
10 Joost
11 Plaxo
10 Powerset
9 eBay
12 Kyte
12 Thoof


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [142]:
# Your Code
cur = db.companies.find({"founded_year":{"$lt":2000},
                  "acquisition.price_amount":{"$gt":1000000000}}).limit(10)

for e in cur:
    print( e["acquisition"]["price_amount"],e["acquisition"]["price_currency_code"],
          "\t", e["founded_year"], e['name'], 
          "\t\t",e['category_code'])

1500000000 USD 	 1998 PayPal 		 finance
7400000000 USD 	 1982 Sun Microsystems 		 enterprise
1200000000 USD 	 1999 Zappos 		 web
1008000000 USD 	 1998 Lastminute 		 web
4300000000 USD 	 1960 Sabre 		 web
1300000000 USD 	 1980 Digitas 		 web
1800000000 USD 	 1996 Omniture 		 advertising
1100000000 USD 	 1996 DoubleClick 		 advertising
8500000000 USD 	 1995 BEA Systems 		 software
6400000000 USD 	 1997 aQuantive 		 web


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [117]:
c = db.companies.find({"acquisition.acquired_year":{"$gt":2010}},
                      {"name":1,"acquisition":1}
                     ).sort([("acquisition.price_amount", DESCENDING)]).limit(10)
for coll in c:
    print(e["acquisition"]["price_currency_code"],
          coll["acquisition"]["price_amount"],
          coll['name'])

USD 39000000000 T-Mobile
USD 18400000000 Goodrich Corporation
USD 6600000000 LSI
USD 6500000000 National Semiconductor
USD 4300000000 Ariba
USD 3700000000 NetLogic Microsystems
USD 3400000000 SuccessFactors
USD 3100000000 Qualcomm Atheros
USD 3000000000 Global Crossing
USD 3000000000 Global Crossing


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [199]:
c= db.companies.find({"founded_year":{"$gt":0}},
                     {"name":1,"founded_year":1}
                     ).sort([("founded_year",ASCENDING)]   
                     ).limit(10)
for e in c:
    print(e["founded_year"],e["name"])

1800 Alstrasoft
1800 SmallWorlds
1800 US Army
1802 DuPont
1833 McKesson
1833 Bachmann Industries
1835 Bertelsmann
1836 Accuity
1839 CENTRA
1840 WeGame


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [204]:
days=[e for e in range(8)]
c = db.companies.find({"founded_day":{"$in":days}}).limit(10)
for e in c:
    print(e["founded_day"], e["name"])

1 Omnidrive
1 Geni
2 Postini
1 Fox Interactive Media
1 Gizmoz
1 Facebook
1 StumbleUpon
1 Helio
1 Joost
1 Babelgum


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [130]:
# Your Code
c=db.companies.find({"category_code":"web", "number_of_employees":{"$gt":4000}}
                   ).sort([("number_of_employees",ASCENDING)])
for e in c:
    print(e["category_code"], e["number_of_employees"], e["name"])

web 4400 Expedia
web 8000 AOL
web 8657 Webkinz
web 10000 Rakuten
web 10000 Los Angeles Times Media Group
web 10000 Groupon
web 13600 Yahoo!
web 15000 eBay
web 15500 Experian


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [135]:
# Your Code
c=db.companies.find({"acquisition.price_amount":{"$gt":10000000},
                     "acquisition.price_currency_code":"EUR"})

for e in c:
    print(e["acquisition"]["price_amount"], e["acquisition"]["price_currency_code"],
          e["category_code"],
          e["number_of_employees"],"\t", e["name"])

31500000 EUR mobile 25 	 ZYB
140000000 EUR mobile None 	 Apertio
40000000 EUR public_relations None 	 Greenfield Online
70000000 EUR web 50 	 Webedia
24000000 EUR None None 	 Wayfinder
70000000 EUR social 170 	 Tuenti Technologies
43400000 EUR ecommerce None 	 BioMed Central


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [214]:
rng= [e for e in range(0,4)]
c=db.companies.find({"acquisition.acquired_month":{"$in":rng}},
                    {"name":1, "acquisition":1}).limit(10)
for e in c:
    print(e["acquisition"]["acquired_month"], e["name"])

c.count()

1 Kyte
2 NetRatings
3 blogTV
1 Livestream
2 iContact
2 Coghead
2 Dailymotion
2 Netvibes
3 Flickr
3 BabyCenter


/home/rh/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  import sys


642

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [141]:
# Your Code
c =db.companies.find({"founded_year":{"$gt":2000,"$lt":2010 },
                     "acquisition.acquired_year":{"$gte":2011}
                     },
                 ).limit(10)
for e in c:
    print(e["founded_year"], e["acquisition"]["acquired_year"] ,e["name"])

2005 2013 Wetpaint
2006 2012 Geni
2004 2012 Digg
2006 2011 Kyte
2005 2011 Jingle Networks
2006 2013 blogTV
2003 2011 delicious
2005 2012 Revision3
2003 2012 iContact
2006 2013 Mashery


### 20. All the companies that have been 'deadpooled' after the third year.

In [229]:
c=db.companies.find({"deadpooled_year":{"$gt":0}, "founded_year":{"$gt":0}},
                   )

for e in c:
    lifetime = e["deadpooled_year"]- e["founded_year"]
    if lifetime > 3:
        print(e["deadpooled_year"], e["founded_year"], "This company was deadpooled after: ",
          (e["deadpooled_year"]- e["founded_year"]),
          "\t\t",  e["name"])

2013 2007 This company was deadpooled after:  6 		 Babelgum
2013 2006 This company was deadpooled after:  7 		 Thoof
2010 2005 This company was deadpooled after:  5 		 Wesabe
2013 2006 This company was deadpooled after:  7 		 Stickam
2008 2004 This company was deadpooled after:  4 		 AllPeers
2012 2006 This company was deadpooled after:  6 		 EQO
2007 2000 This company was deadpooled after:  7 		 AllofMP3
2010 2006 This company was deadpooled after:  4 		 SellABand
2011 2005 This company was deadpooled after:  6 		 Zlio
2012 2006 This company was deadpooled after:  6 		 Jaiku
2009 2003 This company was deadpooled after:  6 		 Coghead
2010 2005 This company was deadpooled after:  5 		 Swivel
2011 2005 This company was deadpooled after:  6 		 Slide
2012 2004 This company was deadpooled after:  8 		 Sway
2012 2006 This company was deadpooled after:  6 		 coComment
2012 2006 This company was deadpooled after:  6 		 Hotelicopter
2012 2007 This company was deadpooled after:  5 		 Ujogo
2009 

2012 2005 This company was deadpooled after:  7 		 Vontoo
2013 2009 This company was deadpooled after:  4 		 Mybandstock
2013 2008 This company was deadpooled after:  5 		 DubMeNow
2010 2001 This company was deadpooled after:  9 		 Microsupply
2012 2008 This company was deadpooled after:  4 		 Engineering Ideas
2013 2007 This company was deadpooled after:  6 		 FanSnap
2012 2007 This company was deadpooled after:  5 		 Karo Internet
2012 2005 This company was deadpooled after:  7 		 Pumant
2012 2007 This company was deadpooled after:  5 		 Tyfish
2011 2007 This company was deadpooled after:  4 		 maufait
2012 2005 This company was deadpooled after:  7 		 Dolphin Geeks
2008 2002 This company was deadpooled after:  6 		 Newmerix
2013 2006 This company was deadpooled after:  7 		 Qoof
2011 2002 This company was deadpooled after:  9 		 Zadby
2008 2001 This company was deadpooled after:  7 		 iKobo
2012 2005 This company was deadpooled after:  7 		 kidthing
2011 2007 This company was deadpo